In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression

In [5]:
df = pd.read_csv(r'/Users/gokul/My Apple/vs_code_practice/retail_sales_forecast/combined_ml_dataset.csv')
df

,Store,Type,Dept,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,IsMarkup,Week_Number,IsHoliday,CPI,Unemployment,Day,Month,Year,Weekly_Sales
0,1,1,1,151315,42.31,2.572,20522.5677,3916.1645,216.3458,19570.7064,8460.3097,0,5,0,211.096358,8.106000,5,2,2010,24924.5000
1,1,1,2,151315,42.31,2.572,20522.5677,3916.1645,216.3458,19570.7064,8460.3097,0,5,0,211.096358,8.106000,5,2,2010,50605.2700
2,1,1,3,151315,42.31,2.572,20522.5677,3916.1645,216.3458,19570.7064,8460.3097,0,5,0,211.096358,8.106000,5,2,2010,13740.1200
3,1,1,4,151315,42.31,2.572,20522.5677,3916.1645,216.3458,19570.7064,8460.3097,0,5,0,211.096358,8.106000,5,2,2010,39954.0400
4,1,1,5,151315,42.31,2.572,20522.5677,3916.1645,216.3458,19570.7064,8460.3097,0,5,0,211.096358,8.106000,5,2,2010,32229.3800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536629,45,2,93,118221,76.06,3.804,212.0200,851.7300,2.0600,10.8800,1864.5700,0,30,0,193.491386,8.590008,26,7,2013,3391.6125
536630,45,2,94,118221,76.06,3.804,212.0200,851.7300,2.0600,10.8800,1864.5700,0,30,0,193.491386,8.590008,26,7,2013,4029.5197
536631,45,2,95,118221,76.06,3.804,212.0200,851.7300,2.0600,10.8800,1864.5700,0,30,0,193.491386,8.590008,26,7,2013,57285.5943
536632,45,2,97,118221,76.06,3.804,212.0200,851.7300,2.0600,10.8800,1864.5700,0,30,0,193.491386,8.590008,26,7,2013,5697.8791


In [6]:
df['IsHoliday'].value_counts()

IsHoliday
0    498045
1     11859
3      8935
4      8934
2      8861
Name: count, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536634 entries, 0 to 536633
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         536634 non-null  int64  
 1   Type          536634 non-null  int64  
 2   Dept          536634 non-null  int64  
 3   Size          536634 non-null  int64  
 4   Temperature   536634 non-null  float64
 5   Fuel_Price    536634 non-null  float64
 6   MarkDown1     536634 non-null  float64
 7   MarkDown2     536634 non-null  float64
 8   MarkDown3     536634 non-null  float64
 9   MarkDown4     536634 non-null  float64
 10  MarkDown5     536634 non-null  float64
 11  IsMarkup      536634 non-null  int64  
 12  Week_Number   536634 non-null  int64  
 13  IsHoliday     536634 non-null  int64  
 14  CPI           536634 non-null  float64
 15  Unemployment  536634 non-null  float64
 16  Day           536634 non-null  int64  
 17  Month         536634 non-null  int64  
 18  Year

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Store,536634.0,22.208621,12.790580,1.000,11.000,22.000000,33.00000,45.000000
Type,536634.0,1.590516,0.666882,1.000,1.000,1.000000,2.00000,3.000000
Dept,536634.0,44.277301,30.527358,1.000,18.000,37.000000,74.00000,99.000000
Size,536634.0,136678.550960,61007.711800,34875.000,93638.000,140167.000000,202505.00000,219622.000000
Temperature,536634.0,58.771762,18.678716,-7.290,45.250,60.060000,73.23000,101.950000
Fuel_Price,536634.0,3.408310,0.430861,2.472,3.041,3.523000,3.74400,4.468000
MarkDown1,536634.0,7508.059820,7604.673913,-2781.450,2992.040,6139.820000,9769.27100,103184.980000
MarkDown2,536634.0,2543.692843,7278.498783,-265.760,57.610,385.678000,1604.52470,104519.540000
MarkDown3,536634.0,1317.385122,9268.229491,-179.260,17.070,104.620000,192.18660,149483.310000
MarkDown4,536634.0,3419.613536,5685.365947,0.000,375.750,1778.830000,3944.04130,67474.850000


In [9]:
df.skew()

Store            0.077555
Type             0.692920
Dept             0.359150
Size            -0.325016
Temperature     -0.267460
Fuel_Price      -0.313840
MarkDown1        3.873663
MarkDown2        6.258987
MarkDown3        9.338325
MarkDown4        4.561776
MarkDown5       52.436475
IsMarkup        13.255411
Week_Number      0.161594
IsHoliday        4.337745
CPI              0.081664
Unemployment     1.029223
Day              0.011278
Month            0.151850
Year             0.130470
Weekly_Sales     3.276856
dtype: float64

## Regression - Sales Prediction

In [11]:
# Defining Independent and dependent variable

x = df.drop('Weekly_Sales', axis = 1)
y = df['Weekly_Sales']

In [12]:
# Splitting the data for train test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(429307, 19) (107327, 19) (429307,) (107327,)


In [16]:
# Function to Compare the metrics for all of the models

models = [('XGB Regressor', XGBRegressor()),
          ('Decision Tree Regressor', DecisionTreeRegressor()),
          ('AdaBoost Regressor' ,AdaBoostRegressor()),
          ('Extra Trees  Regressor', ExtraTreesRegressor()),
          ('Random Forest Regressor', RandomForestRegressor()),
          ('Gradient Boosting Regressor', GradientBoostingRegressor()),
          ('HistGradient Boosting Regressor', HistGradientBoostingRegressor()),
          ('Linear Regression', LinearRegression())
          ]

def ml_regression():

    result = []

    for model_name, model in models:

        model.fit(x_train, y_train)
        y_train_pred = model.predict(x_train)
        y_test_pred = model.predict(x_test)

        metrics = {'Model name': model_name,
                   'Train_mse': round(mean_squared_error(y_train, y_train_pred), 4),
                   'Test_mse': round(mean_squared_error(y_test, y_test_pred), 4),
                   'Train_mae': round(mean_absolute_error(y_train, y_train_pred), 4),
                   'Test_mae': round(mean_absolute_error(y_test, y_test_pred), 4),
                   'Train_R2_score': round(r2_score(y_train, y_train_pred), 4),
                   'Test_R2_score': round(r2_score(y_test, y_test_pred), 4)}
        result.append(metrics)

    score_df = pd.DataFrame(result)

    return(score_df)

In [17]:
# comparing the metrics of all the alogrithms

ml_regression()

,Model name,Train_mse,Test_mse,Train_mae,Test_mae,Train_R2_score,Test_R2_score
0,XGB Regressor,2.343998e+07,2.676462e+07,2850.4459,2930.4897,0.9545,0.9492
1,Decision Tree Regressor,0.000000e+00,2.418139e+07,0.0000,1657.7436,1.0000,0.9541
2,AdaBoost Regressor,7.682686e+08,7.757923e+08,23257.0750,23279.0600,-0.4906,-0.4732
3,Extra Trees Regressor,2.000000e-04,1.004573e+07,0.0000,1166.8385,1.0000,0.9809
4,Random Forest Regressor,1.310562e+06,1.157969e+07,434.9498,1177.7571,0.9975,0.9780
5,Gradient Boosting Regressor,1.237776e+08,1.278111e+08,6796.3472,6824.4418,0.7599,0.7573
6,HistGradient Boosting Regressor,4.259680e+07,4.557224e+07,4023.4460,4058.3078,0.9174,0.9135
7,Linear Regression,4.693252e+08,4.797958e+08,14559.1684,14632.5955,0.0894,0.0889


Comparing all the metrics, Random Forest Algorithm has low error values and high R2 score

Choosing Random Forest Regressor for the further analysis

In [18]:
# taking the random forest model alone

model_rf = RandomForestRegressor().fit(x_train, y_train)

y_test_pred = model_rf.predict(x_test)
y_train_pred = model_rf.predict(x_train)

metrics = {'Train_mse': round(mean_squared_error(y_train, y_train_pred), 4),
            'Test_mse': round(mean_squared_error(y_test, y_test_pred), 4),
            'Train_mae': round(mean_absolute_error(y_train, y_train_pred), 4),
            'Test_mae': round(mean_absolute_error(y_test, y_test_pred), 4),
            'Train_R2_score': round(r2_score(y_train, y_train_pred), 4),
            'Test_R2_score': round(r2_score(y_test, y_test_pred), 4)}

metrics

{'Train_mse': 1345029.4914,
 'Test_mse': 11404936.7994,
 'Train_mae': 435.4109,
 'Test_mae': 1174.322,
 'Train_R2_score': 0.9974,
 'Test_R2_score': 0.9783}

In [20]:
# finding the best Hyper parameters using gridsearch Cross Validation

from sklearn.model_selection import GridSearchCV

param_grid_reg = {'max_depth'        : [2, 5, 10, 20],
                  'min_samples_split': [2, 5, 10],
                  'min_samples_leaf' : [1, 2, 4],
                  'max_features'     : ['sqrt', 'log2', None]}

grid_search_reg = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_grid_reg, cv = 3, n_jobs = -1)
grid_search_reg.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 5, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [21]:
# best parameters from grid search

print(grid_search_reg.best_params_) 
grid_search_reg.best_score_

{'max_depth': 20, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 5}


0.9781732844992534

In [25]:
# passing the best paramseters to check the accuracy for training, testing and overfitting

x1 = df.drop('Weekly_Sales', axis = 1)
y1 = df['Weekly_Sales']

x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size = 0.2, random_state = 42)

model_rf_1 = RandomForestRegressor(max_depth = 20, max_features = None, min_samples_leaf = 2, min_samples_split = 5).fit(x_train, y_train)
y_pred_train = model_rf_1.predict(x_train)
y_pred_test = model_rf_1.predict(x_test)

metrics_1 = {'Train_mse': round(mean_squared_error(y_train, y_train_pred), 4),
            'Test_mse': round(mean_squared_error(y_test, y_test_pred), 4),
            'Train_mae': round(mean_absolute_error(y_train, y_train_pred), 4),
            'Test_mae': round(mean_absolute_error(y_test, y_test_pred), 4),
            'Train_R2_score': round(r2_score(y_train, y_train_pred), 4),
            'Test_R2_score': round(r2_score(y_test, y_test_pred), 4)}

metrics_1

{'Train_mse': 1345029.4914,
 'Test_mse': 11404936.7994,
 'Train_mae': 435.4109,
 'Test_mae': 1174.322,
 'Train_R2_score': 0.9974,
 'Test_R2_score': 0.9783}

Comparing Before and After Hypertuning metrics, both are having more or less same R2-Score meaning more accurate.

In [24]:
# applying the test data for predicting the price

test_data = np.array([[18, 41, 95,120653, 36.33, 3.727, 1692.83, 1.5,7837.15, 579.46, 449.15, 0, 47.1, 3.0, 138.424, 8.243, 23.0, 11, 0.12]])
print('Predicted Weekly Sales Price: $', model_rf.predict(test_data)[0])

Predicted Weekly Sales Price: $ 44011.67089999999


In [26]:
# saving the Regression model by using pickle

import pickle

with open('rf_regression_model.pkl', 'wb') as file:
    pickle.dump(model_rf, file)

In [27]:
# load the pickle model to predict sales price

with open('/Users/gokul/My Apple/vs_code_practice/retail_sales_forecast/rf_regression_model.pkl', 'rb') as f:
    reg_model = pickle.load(f)

In [37]:
print('Predicted Weekly Sales Price: $', reg_model.predict(test_data)[0])

Predicted Weekly Sales Price: $ 44011.67089999999
